In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [154]:
# Create an empty list for the merged data
merged_data = []

# Load datasets
p10umd = pd.read_stata("P10UMD.dta")
factor_ff5 = pd.read_stata("FactorFF5.dta")[['year', 'month', 'mktrf', 'smb', 'hml', 'cma', 'rmw', 'rf', 'yyyymm']]
tsfactor = pd.read_stata("TSFactor.dta")[['year', 'month', 'TSMom', 'yyyymm']]
factor_umd = pd.read_stata("FactorUMD.dta")[['year', 'month', 'umd']]
oos_tsmom = pd.read_stata("oos_tsmom_scs.dta")[['yyyymm', 'tsmom1', 'tsmom2', 'tsmom3', 'tsmom4', 'tsmom5']]

# Merge datasets without the pc
merged_data = pd.merge(p10umd, factor_ff5, on=['year', 'month'], how='inner')
merged_data = pd.merge(merged_data, tsfactor, on=['year', 'month'], how='inner')
merged_data = pd.merge(merged_data, factor_umd, on=['year', 'month'], how='inner')

# Convert 'year' and 'month' columns to int64 in merged_data
merged_data['year'] = merged_data['year'].astype('int64')
merged_data['month'] = merged_data['month'].astype('int64')

# Merge oos_tsmom and perform necessary operations
oos_tsmom['year'] = oos_tsmom['yyyymm'] // 100
oos_tsmom['month'] = oos_tsmom['yyyymm'] % 100
oos_tsmom = oos_tsmom.drop(columns=['yyyymm'])
oos_tsmom = oos_tsmom.sort_values(by=['year', 'month'], ascending=[True, True])

# Multiply columns by 100 and rename
columns_to_multiply = ['tsmom1', 'tsmom2', 'tsmom3', 'tsmom4', 'tsmom5']
columns_to_rename = ['pctsmom1', 'pctsmom2', 'pctsmom3', 'pctsmom4', 'pctsmom5']
for orig_col, new_col in zip(columns_to_multiply, columns_to_rename):
    oos_tsmom[new_col] = oos_tsmom[orig_col] * 100

# Excess returns
for i in range(1, 11):
    merged_data[f'ExcessP{i}'] = merged_data[f'p{i}'] - merged_data['rf']

# Portfolio 11 is high-minus-low
merged_data['ExcessP11'] = merged_data['ExcessP10'] - merged_data['ExcessP1']
    
# Drop the original columns
merged_data1 = pd.merge(merged_data, oos_tsmom, on=['year', 'month'], how='inner')

# Excess returns
for i in range(1, 11):
    merged_data1[f'ExcessP{i}'] = merged_data1[f'p{i}'] - merged_data1['rf']

# Portfolio 11 is high-minus-low
merged_data1['ExcessP11'] = merged_data1['ExcessP10'] - merged_data1['ExcessP1']

# Create an empty list to store regression results
estimates_list = []

print(merged_data)
merged_data.columns
# Other regressions...
# Repeat similar steps for FF5+UMD, FF5+TSMOM, and FF5+PCMOM

# GRS tests (skipping for now)
# ...

# Please note that you would need to adjust the paths to your dataset files accordingly.

        p1    p2    p3    p4    p5    p6    p7    p8    p9   p10  ...  \
0     1.93  2.01  3.41  3.03  3.45  2.52  0.10  3.06  2.06  0.70  ...   
1    -1.32  0.49 -1.13 -0.07 -1.32 -1.93 -1.51 -1.08 -1.72 -0.91  ...   
2     5.77  3.02  3.81  3.03  2.58  2.31  1.81  3.74  2.24  6.37  ...   
3     1.52  1.79  1.31  0.50  0.74 -0.48  1.91  0.71  1.01  0.01  ...   
4    -3.05  0.33  0.83 -0.24 -0.32  0.20  0.00  1.79 -0.18  0.04  ...   
..     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
661 -12.06 -9.91 -8.14 -6.61 -3.51 -5.80 -1.72 -0.48  0.91 -0.31  ...   
662   5.32  5.26  5.89  5.16  0.36  3.03  4.44  1.42  0.34 -3.03  ...   
663  -3.92  2.38  4.04  4.50  3.45  4.79  1.92  1.53  1.01 -0.40  ...   
664   3.88  6.00  6.84  5.82  5.18  3.79  4.48  1.69  2.95  3.43  ...   
665  11.06  5.18  3.22  3.69  1.60  2.85  2.42  2.75  2.12  4.60  ...   

     ExcessP2  ExcessP3  ExcessP4  ExcessP5  ExcessP6  ExcessP7  ExcessP8  \
0        1.71      3.11      2.73      3.15   

Index(['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'year',
       'month', 'mktrf', 'smb', 'hml', 'cma', 'rmw', 'rf', 'yyyymm_x', 'TSMom',
       'yyyymm_y', 'umd', 'ExcessP1', 'ExcessP2', 'ExcessP3', 'ExcessP4',
       'ExcessP5', 'ExcessP6', 'ExcessP7', 'ExcessP8', 'ExcessP9', 'ExcessP10',
       'ExcessP11'],
      dtype='object')

In [156]:
print(merged_data1[['TSMom', 'hml', 'pctsmom1']])

        TSMom   hml  pctsmom1
0   -3.451123 -5.31 -1.270071
1    0.537575  1.14  0.333327
2   -1.119700  2.18  0.803473
3    1.315374  1.74  0.698652
4    2.497214  4.04 -1.334367
..        ...   ...       ...
553  1.944303 -4.99  0.537914
554 -0.636095  6.71 -0.369584
555  0.170413 -2.07  0.147910
556 -0.551105 -1.86  0.307208
557 -0.825733  1.83 -0.134129

[558 rows x 3 columns]


In [158]:
oos_tsmom

,tsmom1,tsmom2,tsmom3,tsmom4,tsmom5,year,month,pctsmom1,pctsmom2,pctsmom3,pctsmom4,pctsmom5
557,-0.012701,-0.000440,0.003060,-0.010772,-0.010661,1973,7,-1.270071,-0.044031,0.306048,-1.077163,-1.066077
556,0.003333,0.004794,-0.003836,0.003890,-0.000330,1973,8,0.333327,0.479411,-0.383611,0.389013,-0.033026
555,0.008035,0.012326,-0.009989,0.000541,-0.001498,1973,9,0.803473,1.232611,-0.998936,0.054115,-0.149806
554,0.006987,0.022122,0.006759,0.000714,-0.003279,1973,10,0.698652,2.212238,0.675942,0.071382,-0.327930
553,-0.013344,-0.004698,0.007924,0.007388,-0.002606,1973,11,-1.334367,-0.469763,0.792408,0.738791,-0.260597
...,...,...,...,...,...,...,...,...,...,...,...,...
4,0.005379,0.006713,-0.001983,0.003886,0.002264,2019,8,0.537914,0.671287,-0.198309,0.388581,0.226372
3,-0.003696,-0.007583,-0.003462,-0.000369,-0.005612,2019,9,-0.369584,-0.758331,-0.346236,-0.036860,-0.561151
2,0.001479,-0.000615,-0.000670,-0.003649,-0.005284,2019,10,0.147910,-0.061492,-0.067011,-0.364900,-0.528394
1,0.003072,0.004403,-0.000078,-0.006257,0.003168,2019,11,0.307208,0.440266,-0.007838,-0.625736,0.316828


In [174]:
# Create an empty list to store regression results
estimatesFF5_list = []

# Explain 10 momentum sorted portfolios with FF5
for i in range(1, 12):
    X = sm.add_constant(merged_data[['mktrf', 'smb', 'hml', 'cma', 'rmw']])
    model = sm.OLS(merged_data[f'ExcessP{i}'], X).fit()
    estimatesFF5_list.append(model)

# Print regression results
for i, est in enumerate(estimatesFF5_list, start=1):
    print(f"\nRegression Results for Excess Portfolio Returns with FF5 - Portfolio {i}:\n")
    print(est.summary())


Regression Results for Excess Portfolio Returns with FF5 - Portfolio 1:

                            OLS Regression Results                            
Dep. Variable:               ExcessP1   R-squared:                       0.683
Model:                            OLS   Adj. R-squared:                  0.680
Method:                 Least Squares   F-statistic:                     283.8
Date:                Mon, 19 Feb 2024   Prob (F-statistic):          9.62e-162
Time:                        21:05:55   Log-Likelihood:                -1954.6
No. Observations:                 666   AIC:                             3921.
Df Residuals:                     660   BIC:                             3948.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

In [241]:
# Print header
print("Results for FF5 Model:\n")

# Create an empty list to store results
resultsFF5_list = []

# Append results for FF5 + UMD
for i, est_FF5 in enumerate(estimatesFF5_list, start=1):
    result_row = {
        'Decile': f'{i}',
        'FF5_Alpha': f'{est_FF5.params["const"]:.2f}',
    }
    t_stat_row = {
        'Decile': f'', 
        'FF5_Alpha': f'({est_FF5.tvalues["const"]:.2f})',
    }
    resultsFF5_list.append(result_row)
    resultsFF5_list.append(t_stat_row)

# Convert the list of results to a DataFrame
resultsFF5_df = pd.DataFrame(resultsFF5_list)

# Display the results DataFrame
print(resultsFF5_df)

Results for FF5 Model:

   Decile FF5_Alpha
0       1     -0.75
1           (-4.05)
2       2     -0.35
3           (-2.73)
4       3     -0.20
5           (-1.90)
6       4     -0.16
7           (-1.93)
8       5     -0.16
9           (-2.45)
10      6     -0.13
11          (-2.05)
12      7     -0.12
13          (-1.94)
14      8      0.04
15           (0.62)
16      9      0.08
17           (1.08)
18     10      0.57
19           (4.82)
20     11      1.33
21           (4.91)


In [243]:
# Create an empty list to store regression results
estimatesUMD_list = []

# Explain 10 momentum sorted portfolios with FF5 + UMD
for i in range(1, 12):
    X = sm.add_constant(merged_data[['mktrf', 'smb', 'hml', 'cma', 'rmw', 'umd']])
    model = sm.OLS(merged_data[f'ExcessP{i}'], X).fit()
    estimatesUMD_list.append(model)

# Print regression results
for i, est in enumerate(estimatesUMD_list, start=1):
    print(f"\nRegression Results for Excess Portfolio Returns with FF5 + UMD - Portfolio {i}:\n")
    print(est.summary())


Regression Results for Excess Portfolio Returns with FF5 + UMD - Portfolio 1:

                            OLS Regression Results                            
Dep. Variable:               ExcessP1   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     939.0
Date:                Mon, 19 Feb 2024   Prob (F-statistic):          5.55e-319
Time:                        21:39:03   Log-Likelihood:                -1585.3
No. Observations:                 666   AIC:                             3185.
Df Residuals:                     659   BIC:                             3216.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

In [245]:
# Print header
print("Results for FF5 + UMD Model:\n")

# Create an empty list to store results
resultsUMD_list = []

# Append results for FF5 + UMD
for i, est_umd in enumerate(estimatesUMD_list, start=1):
    result_row = {
        'Decile': f'{i}',
        'UMD_Alpha': f'{est_umd.params["const"]:.2f}',
        'UMD_Coefficient': f'{est_umd.params["umd"]:.2f}'
    }
    t_stat_row = {
        'Decile': f'', 
        'UMD_Alpha': f'({est_umd.tvalues["const"]:.2f})',
        'UMD_Coefficient': f'({est_umd.tvalues["umd"]:.2f})'
    }
    resultsUMD_list.append(result_row)
    resultsUMD_list.append(t_stat_row)

# Convert the list of results to a DataFrame
resultsUMD_df = pd.DataFrame(resultsUMD_list)

# Display the results DataFrame
print(resultsUMD_df)

Results for FF5 + UMD Model:

   Decile UMD_Alpha UMD_Coefficient
0       1     -0.10           -0.93
1           (-0.94)        (-36.59)
2       2      0.13           -0.70
3            (2.08)        (-46.76)
4       3      0.18           -0.54
5            (2.92)        (-38.35)
6       4      0.07           -0.33
7            (1.20)        (-22.77)
8       5     -0.04           -0.17
9           (-0.65)        (-12.30)
10      6     -0.09           -0.05
11          (-1.46)         (-3.52)
12      7     -0.16            0.07
13          (-2.72)          (4.73)
14      8     -0.11            0.22
15          (-2.05)         (16.96)
16      9     -0.14            0.33
17          (-2.46)         (23.85)
18     10      0.17            0.57
19           (2.32)         (32.93)
20     11      0.27            1.51
21           (2.43)         (56.81)


In [249]:
# Create an empty list to store regression results
estimatesFMOMind_list = []

# Explain 10 momentum sorted portfolios with FF5 + MOMind
for i in range(1, 12):
    X = sm.add_constant(merged_data[['mktrf', 'smb', 'hml', 'cma', 'rmw', 'TSMom']])
    model = sm.OLS(merged_data[f'ExcessP{i}'], X).fit()
    estimatesFMOMind_list.append(model)

# Print regression results
for i, est in enumerate(estimatesFMOMind_list, start=1):
    print(f"\nRegression Results for Excess Portfolio Returns with FF5 + FMOMind - Portfolio {i}:\n")
    print(est.summary())


Regression Results for Excess Portfolio Returns with FF5 + FMOMind - Portfolio 1:

                            OLS Regression Results                            
Dep. Variable:               ExcessP1   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     447.2
Date:                Mon, 19 Feb 2024   Prob (F-statistic):          1.55e-228
Time:                        21:42:25   Log-Likelihood:                -1796.0
No. Observations:                 666   AIC:                             3606.
Df Residuals:                     659   BIC:                             3637.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------

In [259]:
# Print header
print("Results for FF5 + FMOMind Model:\n")

# Create an empty list to store results
resultsFMOMind_list = []

# Append results for FF5 + FMOMind
for i, est_FMOMind in enumerate(estimatesFMOMind_list, start=1):
    result_row = {
        'Decile': f'{i}',
        'FMOMind_Alpha': f'{est_FMOMind.params["const"]:.2f}',
        'FMOMind_Coefficient': f'{est_FMOMind.params["TSMom"]:.2f}'
    }
    t_stat_row = {
        'Decile': f'', 
        'FMOMind_Alpha': f'({est_FMOMind.tvalues["const"]:.2f})',
        'FMOMind_Coefficient': f'({est_FMOMind.tvalues["TSMom"]:.2f})'
    }
    resultsFMOMind_list.append(result_row)
    resultsFMOMind_list.append(t_stat_row)

# Convert the list of results to a DataFrame
resultsFMOMind_df = pd.DataFrame(resultsFMOMind_list)

# Display the results DataFrame
print(resultsFMOMind_df)

Results for FF5 + FMOMind Model:

   Decile FMOMind_Alpha FMOMind_Coefficient
0       1         -0.04               -2.46
1               (-0.28)            (-20.06)
2       2          0.16               -1.78
3                (1.54)            (-21.26)
4       3          0.17               -1.30
5                (1.93)            (-17.78)
6       4          0.12               -0.95
7                (1.69)            (-16.70)
8       5         -0.02               -0.47
9               (-0.39)             (-9.07)
10      6         -0.07               -0.22
11              (-1.02)             (-4.26)
12      7         -0.14                0.09
13              (-2.32)              (1.83)
14      8         -0.09                0.44
15              (-1.34)              (8.42)
16      9         -0.11                0.66
17              (-1.45)             (11.04)
18     10          0.16                1.42
19               (1.59)             (17.21)
20     11          0.20                3.8

In [261]:
# Create an empty list to store regression results
estimatesFMOMpc_list = []

# Explain 10 momentum sorted portfolios with FF5 + UMD
for i in range(1, 12):
    X = sm.add_constant(merged_data1[['mktrf', 'smb', 'hml', 'cma', 'rmw', 'pctsmom1']])
    model = sm.OLS(merged_data1[f'ExcessP{i}'], X).fit()
    estimatesFMOMpc_list.append(model)

# Print regression results
for i, est in enumerate(estimatesFMOMpc_list, start=1):
    print(f"\nRegression Results for Excess Portfolio Returns with FF5 + FMOMpc - Portfolio {i}:\n")
    print(est.summary())


Regression Results for Excess Portfolio Returns with FF5 + FMOMpc - Portfolio 1:

                            OLS Regression Results                            
Dep. Variable:               ExcessP1   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     273.6
Date:                Mon, 19 Feb 2024   Prob (F-statistic):          1.21e-161
Time:                        21:49:04   Log-Likelihood:                -1589.1
No. Observations:                 558   AIC:                             3192.
Df Residuals:                     551   BIC:                             3222.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

In [263]:
# Print header
print("Results for FF5 + FMOMpc Model:\n")

# Create an empty list to store results
resultsFMOMpc_list = []

# Append results for FF5 + FMOMind
for i, est_FMOMpc in enumerate(estimatesFMOMpc_list, start=1):
    result_row = {
        'Decile': f'{i}',
        'FMOMpc_Alpha': f'{est_FMOMpc.params["const"]:.2f}',
        'FMOMpc_Coefficient': f'{est_FMOMpc.params["pctsmom1"]:.2f}'
    }
    t_stat_row = {
        'Decile': f'', 
        'FMOMpc_Alpha': f'({est_FMOMpc.tvalues["const"]:.2f})',
        'FMOMpc_Coefficient': f'({est_FMOMpc.tvalues["pctsmom1"]:.2f})'
    }
    resultsFMOMpc_list.append(result_row)
    resultsFMOMpc_list.append(t_stat_row)

# Convert the list of results to a DataFrame
resultsFMOMpc_df = pd.DataFrame(resultsFMOMpc_list)

# Display the results DataFrame
print(resultsFMOMpc_df)

Results for FF5 + FMOMpc Model:

   Decile FMOMpc_Alpha FMOMpc_Coefficient
0       1        -0.02              -3.65
1              (-0.09)           (-12.95)
2       2         0.18              -2.66
3               (1.35)           (-14.05)
4       3         0.20              -2.06
5               (1.83)           (-13.16)
6       4         0.16              -1.43
7               (1.93)           (-11.67)
8       5         0.02              -0.78
9               (0.22)            (-7.49)
10      6        -0.05              -0.41
11             (-0.65)            (-3.90)
12      7        -0.12               0.06
13             (-1.70)             (0.58)
14      8        -0.10               0.69
15             (-1.44)             (6.68)
16      9        -0.14               0.94
17             (-1.61)             (7.56)
18     10         0.01               2.38
19              (0.05)            (14.26)
20     11         0.02               6.03
21              (0.09)            (15.84)


In [172]:
print(merged_data1)

            p1         p2     p3     p4     p5     p6    p7     p8    p9  \
0    20.969999  12.990000  15.15  11.50  11.28  10.15  7.18   3.08  2.84   
1    -7.150000  -5.280000  -5.37  -4.23  -2.42  -3.16 -3.07  -2.11 -4.12   
2    11.010000  13.150000  14.97  11.23  10.41  12.27  5.49  -1.18  3.78   
3    -5.680000  -5.570000  -3.59  -1.94  -2.87  -4.33  0.95  -0.04 -0.58   
4   -23.160000 -22.440001 -19.82 -17.58 -11.33 -13.70 -9.58 -10.90 -9.16   
..         ...        ...    ...    ...    ...    ...   ...    ...   ...   
553 -12.060000  -9.910000  -8.14  -6.61  -3.51  -5.80 -1.72  -0.48  0.91   
554   5.320000   5.260000   5.89   5.16   0.36   3.03  4.44   1.42  0.34   
555  -3.920000   2.380000   4.04   4.50   3.45   4.79  1.92   1.53  1.01   
556   3.880000   6.000000   6.84   5.82   5.18   3.79  4.48   1.69  2.95   
557  11.060000   5.180000   3.22   3.69   1.60   2.85  2.42   2.75  2.12   

      p10  ...    tsmom1    tsmom2    tsmom3    tsmom4    tsmom5  pctsmom1  \
0    2.30

In [267]:
# Assuming 'Portfolio' is the common column for merging
merged_results = pd.merge(resultsFF5_df, resultsUMD_df, on='Decile')
merged_results = pd.merge(merged_results, resultsFMOMind_df, on='Decile')
merged_results = pd.merge(merged_results, resultsFMOMpc_df, on='Decile')

# Display the merged results DataFrame
print(merged_results)

      Decile FF5_Alpha UMD_Alpha UMD_Coefficient FMOMind_Alpha  \
0          1     -0.75     -0.10           -0.93         -0.04   
1              (-4.05)   (-0.94)        (-36.59)       (-0.28)   
2              (-4.05)   (-0.94)        (-36.59)       (-0.28)   
3              (-4.05)   (-0.94)        (-36.59)       (-0.28)   
4              (-4.05)   (-0.94)        (-36.59)       (-0.28)   
...      ...       ...       ...             ...           ...   
14647      7     -0.12     -0.16            0.07         -0.14   
14648      8      0.04     -0.11            0.22         -0.09   
14649      9      0.08     -0.14            0.33         -0.11   
14650     10      0.57      0.17            0.57          0.16   
14651     11      1.33      0.27            1.51          0.20   

      FMOMind_Coefficient FMOMpc_Alpha FMOMpc_Coefficient  
0                   -2.46        -0.02              -3.65  
1                (-20.06)      (-0.09)           (-12.95)  
2                (-20.06)  

In [271]:
# Drop 'Decile' column from resultsUMD_df, resultsFMOMind_df, and resultsFMOMpc_df
resultsUMD_df = resultsUMD_df.drop(columns=['Decile'])
resultsFMOMind_df = resultsFMOMind_df.drop(columns=['Decile'])
resultsFMOMpc_df = resultsFMOMpc_df.drop(columns=['Decile'])

# Concatenate dataframes horizontally
merged_results = pd.concat([resultsFF5_df, resultsUMD_df, resultsFMOMind_df, resultsFMOMpc_df], axis=1)

# Display the merged results DataFrame
print(merged_results)

   Decile FF5_Alpha UMD_Alpha UMD_Coefficient FMOMind_Alpha  \
0       1     -0.75     -0.10           -0.93         -0.04   
1           (-4.05)   (-0.94)        (-36.59)       (-0.28)   
2       2     -0.35      0.13           -0.70          0.16   
3           (-2.73)    (2.08)        (-46.76)        (1.54)   
4       3     -0.20      0.18           -0.54          0.17   
5           (-1.90)    (2.92)        (-38.35)        (1.93)   
6       4     -0.16      0.07           -0.33          0.12   
7           (-1.93)    (1.20)        (-22.77)        (1.69)   
8       5     -0.16     -0.04           -0.17         -0.02   
9           (-2.45)   (-0.65)        (-12.30)       (-0.39)   
10      6     -0.13     -0.09           -0.05         -0.07   
11          (-2.05)   (-1.46)         (-3.52)       (-1.02)   
12      7     -0.12     -0.16            0.07         -0.14   
13          (-1.94)   (-2.72)          (4.73)       (-2.32)   
14      8      0.04     -0.11            0.22         -

**Calculate average alphas**

In [326]:
# Create empty lists to store absolute mean alpha estimates for each model
alpha_means_UMD = []
alpha_means_FMOMind = []
alpha_means_FMOMpc = []

# Calculate mean alpha for each portfolio and store absolute values
for i, est_umd in enumerate(estimatesUMD_list, start=1):
    alpha_mean_umd = np.abs(est_umd.params['const']).mean()
    alpha_means_UMD.append(alpha_mean_umd)

for i, est_fmomind in enumerate(estimatesFMOMind_list, start=1):
    alpha_mean_fmomind = np.abs(est_fmomind.params['const']).mean()
    alpha_means_FMOMind.append(alpha_mean_fmomind)

for i, est_fmompc in enumerate(estimatesFMOMpc_list, start=1):
    alpha_mean_fmompc = np.abs(est_fmompc.params['const']).mean()
    alpha_means_FMOMpc.append(alpha_mean_fmompc)

# Calculate the overall absolute mean alphas excluding the last row
Avg_alpha_UMD = np.mean(alpha_means_UMD[:-1])
Avg_alpha_FMOMind = np.mean(alpha_means_FMOMind[:-1])
Avg_alpha_FMOMpc = np.mean(alpha_means_FMOMpc[:-1])

# Create a DataFrame for average alphas
avg_alphas_df = pd.DataFrame({
    'Model': ['FF5', 'UMD', 'FMOMind', 'FMOMpc'],
    'Avg_alpha': [Avg_alpha_FF5, Avg_alpha_UMD, Avg_alpha_FMOMind, Avg_alpha_FMOMpc]
})

# Display the average alphas DataFrame
print(avg_alphas_df)

     Model  Avg_alpha
0      FF5   0.257050
1      UMD   0.120639
2  FMOMind   0.108118
3   FMOMpc   0.098437
